In [1]:
from __future__ import annotations

from pathlib import Path

import numpy as np
import pandas as pd

In [2]:
jkp_data = pd.read_csv(Path("../../data/crsp_raw") / "crsp_80s.csv")
jkp_data = jkp_data.rename(columns={c: c.lower() for c in jkp_data.columns})
jkp_data = jkp_data.dropna(subset=["permno"])
jkp_data["permno"] = jkp_data["permno"].astype(int)
jkp_data["date"] = pd.to_datetime(jkp_data["date"])
jkp_data = jkp_data.sort_values(["date", "permno"])
jkp_data = jkp_data.drop_duplicates(subset=["date", "permno"])
jkp_data = jkp_data.set_index(["date", "permno"])
jkp_data.head()

/var/folders/sr/lzvmv9j54ks_jl4x99t250r00000gn/T/ipykernel_20609/2717679415.py:1: DtypeWarning: Columns (5,6,9,18,19,20,22,24,38,40,49,57) have mixed types. Specify dtype option on import or set low_memory=False.
  jkp_data = pd.read_csv(Path("../../data/crsp_raw") / "crsp_80s.csv")


nameendt  shrcd  exchcd siccd    ncusip ticker  \
date       permno                                                  
1980-01-02 10006       NaN     10       1  3743     80010    ACF   
           10057       NaN     11       1  3541  00462610    AMT   
           10058       NaN     10       3  5099  00077410   ABKC   
           10103       NaN     10       3  3496  00087410   ACSC   
           10137       NaN     11       1  4911  01741110    AYP   

                                        comnam shrcls tsymbol  naics  ...  \
date       permno                                                     ...   
1980-01-02 10006          A C F INDUSTRIES INC    NaN     NaN    NaN  ...   
           10057           ACME CLEVELAND CORP    NaN     NaN    NaN  ...   
           10058      A B K C O INDUSTRIES INC    NaN     NaN    NaN  ...   
           10103          A C S INDUSTRIES INC    NaN     NaN    NaN  ...   
           10137   ALLEGHENY POWER SYSTEMS INC    NaN     NaN    NaN  ...   

                  cfacpr cfacshr openprc  numtrd       retx    vwretd  \
date       permno                                                       
1980-01-02 10006     1.0     1.0     NaN     NaN   0.007299 -0.020089   
           10057     1.0     1.0     NaN     NaN  -0.004785 -0.020089   
           10058     1.0     1.0     NaN     NaN   0.000000 -0.020089   
           10103     1.0     1.0     NaN     NaN   0.166667 -0.020089   
           10137     2.0     2.0     NaN     NaN  -0.024194 -0.020089   

                     vwretx    ewretd    ewretx    sprtrn  
date       permno                                          
1980-01-02 10006  -0.020121 -0.011686 -0.011707 -0.020196  
           10057  -0.020121 -0.011686 -0.011707 -0.020196  
           10058  -0.020121 -0.011686 -0.011707 -0.020196  
           10103  -0.020121 -0.011686 -0.011707 -0.020196  
           10137  -0.020121 -0.011686 -0.011707 -0.020196  

[5 rows x 61 columns]

In [28]:
mapping = jkp_data.reset_index()[["permno", "comnam"]].drop_duplicates()
mapping = mapping.set_index("permno")

In [29]:
mapping.to_csv(Path("../../data/output") / "crsp_mapping.csv")

In [5]:
CRSP_IGNORED = [-66, -77, -88, -99]

jkp_data = jkp_data[
    (jkp_data["ret"] != CRSP_IGNORED[0])
    & (jkp_data["ret"] != CRSP_IGNORED[1])
    & (jkp_data["ret"] != CRSP_IGNORED[2])
    & (jkp_data["ret"] != CRSP_IGNORED[3])
]

In [6]:
jkp_data["ret"] = jkp_data["ret"].replace("C", np.nan).astype(float)

In [7]:
# jkp_data["prc"] = np.abs(jkp_data["prc"]) / jkp_data["cfacpr"].ffill().fillna(1).replace(0, 1.0)

In [8]:
jkp_data["mktcap"] = jkp_data["shrout"] * 1_000 * jkp_data["prc"]

In [9]:
dolvol = jkp_data.reset_index().pivot(index="date", columns="permno", values="mktcap")

In [10]:
dolvol = dolvol.resample("ME").last()

In [11]:
N_LARGEST = 20

presence_matrix = dolvol.apply(lambda x: x >= x.nlargest(N_LARGEST).min(), axis=1).astype(float)
presence_matrix[presence_matrix == 0] = np.nan

In [12]:
presence_matrix = presence_matrix.dropna(axis=1, how="all")
presence_matrix.shape

(540, 96)

In [13]:
pivoted_returns = (
    jkp_data.loc[jkp_data.index.get_level_values("permno").isin(presence_matrix.columns)]
    .reset_index()
    .pivot_table(index="date", columns="permno", values="ret")
)

In [14]:
last_selection = presence_matrix.iloc[-1]
last_selection = last_selection[last_selection.notna()].index

In [15]:
last_selection

Index([10104, 10107, 11850, 13407, 14542, 14593, 18163, 47896, 50876, 55976,
       83443, 84788, 86580, 87055, 90319, 91233, 92611, 92655, 93002, 93436],
      dtype='int64', name='permno')

In [16]:
# df_data = jkp_data.reset_index()
# df_data[df_data["permno"].isin(last_selection)]["comnam"].drop_duplicates()

In [17]:
full_df = pivoted_returns

In [18]:
valid_cols = presence_matrix.columns.intersection(full_df.columns)
len(valid_cols)

96

In [19]:
presence_matrix = presence_matrix.reset_index()
presence_matrix["date"] = pd.to_datetime(presence_matrix["date"])
presence_matrix = presence_matrix.set_index("date")

In [20]:
presence_matrix

permno,10078,10104,10107,10147,10401,10604,11042,11081,11308,11471,...,89393,89525,90215,90319,91233,92602,92611,92655,93002,93436
date,,,,,,,,,,,,,,,,,,,,,
1980-01-31,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-02-29,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-31,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-04-30,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-05-31,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,1.0,NaN,1.0,1.0,1.0,1.0
2024-09-30,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,1.0,NaN,1.0,1.0,1.0,1.0
2024-10-31,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,1.0,NaN,1.0,1.0,1.0,1.0


In [21]:
presence_matrix = presence_matrix.resample("D").ffill()

In [22]:
merged_index = full_df.merge(
    presence_matrix, left_index=True, right_index=True, how="inner"
).index

In [24]:
full_df = full_df.loc[merged_index]
presence_matrix = presence_matrix.loc[merged_index]
full_df.shape, presence_matrix.shape

((11324, 96), (11324, 96))

In [25]:
full_df[valid_cols].to_csv(Path("../../data/output") / f"top{N_LARGEST}_data.csv")

In [26]:
presence_matrix[valid_cols].to_csv(Path("../../data/output") / f"top{N_LARGEST}_presence_matrix.csv")

In [27]:
pd.DataFrame(valid_cols).to_csv(
    Path("../../data/output") / f"top{N_LARGEST}_stocks_list.csv",
    index=False,
)